In [1]:
import boto3
import awswrangler as wr
import pandas as pd
from tqdm import tqdm
accessKeys = pd.read_csv("../../quant-bears_accessKeys.csv")
session = boto3.Session(
	aws_access_key_id=accessKeys.loc[0, "Access key ID"],
	aws_secret_access_key=accessKeys.loc[0, "Secret access key"]
)

s3_collection_path = "s3://quant-bears-data-collection/raw-data/"
s3_price_collection_path = "s3://quant-bears-data-collection/raw-resolved-price/"

## Load Data

In [2]:
data_sources = ["seekingAlpha.seekingAlphaBulkMetrics", "gurufocus"]
sources_dict = dict((source, wr.s3.list_objects(s3_collection_path + source + "/", boto3_session=session)) for source in data_sources)
df_dict = {}
for source in data_sources:
	dfs = []
	print(source)
	for path in tqdm(sources_dict[source]):
		new_df = wr.s3.read_parquet(path, boto3_session=session)
		new_df["date"] = path.split("/")[-1].split(".")[0]
		dfs.append(new_df)

	df_dict[source] = pd.concat(dfs, axis = 0)
joined_df = pd.concat([df.set_index(["date", "ticker"]) for df in df_dict.values()], axis = 1)
joined_df.shape

seekingAlpha.seekingAlphaBulkMetrics


  0%|          | 0/8 [00:00<?, ?it/s]/var/folders/tq/51dxx6813y7g8dg44h973dmr0000gn/T/ipykernel_81092/716229386.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df["date"] = path.split("/")[-1].split(".")[0]
 12%|█▎        | 1/8 [00:01<00:07,  1.09s/it]/var/folders/tq/51dxx6813y7g8dg44h973dmr0000gn/T/ipykernel_81092/716229386.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df["date"] = path.split("/")[-1].split(".")[0]
 25%|██▌       | 2/8 [00:02<00:07,  1.29s/it]/var/folders/tq/51dxx6813y7g8dg44h973dmr0000gn/T/ip

gurufocus


100%|██████████| 8/8 [00:05<00:00,  1.52it/s]


(12708, 337)

## Preprocess

#### Step 1: Drop all rows without "primary_price" feature

In [5]:
step1_df = joined_df[~joined_df["primary_price"].isna()]
step1_df.shape

(12700, 337)

#### Step 2: Drop all columns with >80% NaNs

In [8]:
nan_pct_per_col = step1_df.isna().sum() / step1_df.shape[0]
nan_cols = step1_df.columns[nan_pct_per_col > .8]
step2_df = step1_df.drop(nan_cols, axis = 1)
step2_df.shape

(12700, 294)

#### Step 3: Convert category columns to one-hot columns

In [48]:
float_df = step2_df.select_dtypes([int, float])
object_df = step2_df.select_dtypes("string")
onehot_df = pd.get_dummies(object_df).astype("float")
step3_df = pd.concat([float_df, onehot_df], axis = 1)
step3_df.shape

(12700, 358)

#### Step 4: Median and Zero Imputation on columns with stddev effect of <10%

In [49]:
def fill_median_or_zero_by_thresh(df: pd.DataFrame, thresh: float):
	def stddev_diff(col):
		stddev_median_diff = col.fillna(col.median()).std() / col.std()
		stddev_zero_diff = col.fillna(0.).std() / col.std()
		return (stddev_median_diff, stddev_zero_diff, "median" if stddev_median_diff <= stddev_zero_diff else "zero")
	
	float_df = df.select_dtypes([int, float])
	stddev_diffs = float_df.apply(stddev_diff).rename(index = {0: "median_fill", 1: "zero_fill", 2: "method"})
	dist_from_1 = (1. - stddev_diffs.T[["median_fill", "zero_fill"]]).abs()
	dist_from_1["min_diff"] = dist_from_1.min(axis = 1)
	passed_diffs = stddev_diffs.T[dist_from_1["min_diff"] <= thresh]

	def apply_imputation(col: pd.Series):
		if col.name in passed_diffs.index:
			method = passed_diffs.loc[col.name]["method"]
			if method == "zero":
				return col.fillna(0.)
			elif method == "median":
				return col.fillna(col.median())
		return col
	
	return df.apply(apply_imputation)

In [50]:
float_df = step3_df.select_dtypes([int, float])
object_df = step3_df.select_dtypes("object")

print("NaNs before imputation:")
print(float_df.isna().sum().sum())

imputed_float_df = fill_median_or_zero_by_thresh(float_df, .1)

print("NaNs after imputation:")
print(imputed_float_df.isna().sum().sum())

step4_df = pd.concat([imputed_float_df, object_df], axis = 1)
step4_df.shape

NaNs before imputation:
652309
NaNs after imputation:
405045


(12700, 358)

#### Step 5: Predict remaining columns

In [51]:
from sklearn.linear_model import LinearRegression

step5_df = step4_df.copy()
nan_cols = step4_df.columns[step4_df.isna().sum() > 0]
X = step4_df.drop(nan_cols, axis = 1)
for col in tqdm(nan_cols):
	y_train = step4_df[col].dropna()
	mask = X.index.isin(y_train.index)
	X_train = X[mask]
	X_test = X[~mask]

	linreg = LinearRegression()

	linreg.fit(X_train, y_train)
	y_pred = linreg.predict(X_test)
	descriptions = y_train.describe()

	y_pred = y_pred.clip(min = descriptions["min"], max=descriptions["max"])
	pred_col = pd.Series(y_pred, index = X_test.index).reindex(X.index)
	step5_df[col] = step4_df[col].where(~step4_df[col].isna(), pred_col)


  0%|          | 0/89 [00:00<?, ?it/s]

100%|██████████| 89/89 [00:35<00:00,  2.50it/s]


In [54]:
# No NaNs!
step5_df.isna().sum().sum()

0